### Elementary Probability and Information Theory 


#### Instructions and Hints:

* This notebook loads some data into a `pandas` dataframe, then does a small amount of preprocessing. Make sure your data can load by stepping through all of the cells up until question 1. 
* Most of the questions require you to write some code. In many cases, you will write some kind of probability function like we did in class using the data. 
* Some of the questions only require you to write answers, so be sure to change the cell type to markdown or raw text
* Don't worry about normalizing the text this time (e.g., lowercase, etc.). Just focus on probabilies. 
* Most questions can be answered in a single cell, but you can make as many additional cells as you need. 
* Follow the instructions on the corresponding assignment Trello card for submitting your assignment. 

In [38]:
import pandas as pd 

data = pd.read_csv('pnp-train.txt',delimiter='\t',encoding='latin-1', # utf8 encoding didn't work for this
                  names=['type','name']) # supply the column names for the dataframe

# this next line creates a new column with the lower-cased first word
data['first_word'] = data['name'].map(lambda x: x.lower().split()[0])
smoother = 0.00001

In [2]:
data[:10]

,type,name,first_word
0,drug,Dilotab,dilotab
1,movie,Beastie Boys: Live in Glasgow,beastie
2,person,Michelle Ford-Eriksson,michelle
3,place,Ramsbury,ramsbury
4,place,Market Bosworth,market
5,drug,Cyanide Antidote Package,cyanide
6,person,Bill Johnson,bill
7,place,Ettalong,ettalong
8,movie,The Suicide Club,the
9,place,Pézenas,pézenas


In [3]:
data.describe()

,type,name,first_word
count,21001,21001,21001
unique,5,20992,13703
top,movie,Ash,the
freq,6262,2,635


## 1. Write a probability function/distribution $P(T)$ over the types. 

Hints:

* The Counter library might be useful: `from collections import Counter`
* Write a function `def P(T='')` that returns the probability of the specific value for T
* You can access the types from the dataframe by calling `data['type']`

In [181]:
from collections import Counter

count_type = Counter(data['type'])

def P(T):
    global count_type
    global data
    
    return float(count_type[T]/len(data['type']))

## 2. What is `P(T='movie')` ?

In [183]:
P(T='movie')

0.29817627732012764

## 3. Show that your probability distribution sums to one.

In [184]:
import numpy as np

np.sum([P(T=x) for x in set(data['type'])])

1.0

## 4. Write a joint distribution using the type and the first word of the name

Hints:

* The function is $P2(T,W_1)$
* You will need to count up types AND the first words, for example: ('person','bill)
* Using the [itertools.product](https://docs.python.org/2/library/itertools.html#itertools.product) function was useful for me here

In [185]:
p_w_t_estimate = {}
p_w_t_totals = {}

for t in set(data.type):
    sub_frame = data[data.type == t]
    sub_words = [word for word in sub_frame.first_word]
    p_w_t_estimate[t] = Counter(sub_words)
    p_w_t_totals[t] = len(sub_words)

def P2(T='', W1=''):
    return p_w_t_estimate[T][W1] / sum(p_w_t_totals.values())

## 5. What is P2(T='person', W1='bill')? What about P2(T='movie',W1='the')?

In [186]:
P2(T='person', W1='bill')

0.00047616780153326033

In [187]:
P2(T='movie', W1='the')

0.02747488214846912

## 6. Show that your probability distribution P(T,W1) sums to one.

In [188]:
sum([P2(t, word) for t in set(data.type) for word in set(data[data.type == t].first_word)])

0.9999999999997764

## 7. Make a new function Q(T) from marginalizing over P(T,W1) and make sure that Q(T) sums to one.

Hints:

* Your Q function will call P(T,W1)
* Your check for the sum to one should be the same answer as Question 3, only it calls Q instead of P.

In [189]:
def Q(T):
    return np.sum([P2(T, W1=w) for w in set(data.first_word)])

In [190]:
Q('movie')

0.29817627732012764

In [191]:
sum([Q(t) for t in set(data.type)])

1.0

## 8. What is the KL Divergence of your Q function and your P function for Question 1?

* Even if you know the answer, you still need to write code that computes it.

In [192]:
#step through each of the possible values, steps through 5 different things
import math
(P('movie') * math.log(P('movie')/Q('movie'))) + (P('drug') * math.log(P('drug')/Q('drug'))) + (P('person') * math.log(P('person')/Q('person'))) + (P('place') * math.log(P('place')/Q('place'))) + (P('company') * math.log(P('company')/Q('company')))
#close enough to zero

-3.9722945607511195e-17

## 9. Convert from P(T,W1) to P(W1|T) 

Hints:

* Just write a comment cell, no code this time. 
* Note that $P(T,W1) = P(W1,T)$

(try to use markdown math formating, answer in this cell)

## 10. Write a function `Pwt` (that calls the functions you already have) to compute $P(W_1|T)$.

* This will be something like the multiplication rule, but you may need to change something

In [193]:
def Pwt(W1,T):
    if W1 not in p_w_t_estimate[T]: return float(smoother)
    return p_w_t_estimate[T][W1] / p_w_t_totals[T]

## 11. What is P(W1='the'|T='movie')?

In [194]:
Pwt(W1='the',T='movie')

0.09214308527626956

## 12. Use Baye's rule to convert from P(W1|T) to P(T|W1). Write a function Ptw to reflect this. 

Hints:

* Call your other functions.
* You may need to write a function for P(W1) and you may need a new counter for `data['first_word']`

In [202]:
count_word = Counter(data['first_word'])

def Pw(W1):
    global count_word
    global data
    return float(count_word[W1]/(len(data['first_word'])))

def Ptw(T,W1):
    return Pwt(W1,T) * P(T) / Pw(W1)

In [203]:
Ptw(T='movie',W1='the')

0.9086614173228347

## 13 
### What is P(T='movie'|W1='the')? 
### What about P(T='person'|W1='the')?
### What about P(T='drug'|W1='the')?
### What about P(T='place'|W1='the')
### What about P(T='company'|W1='the')

In [205]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [206]:
Ptw(T='person',W1='the')

6.0409448818897646e-05

In [207]:
Ptw(T='drug',W1='the')

7.921259842519686e-05

In [208]:
Ptw(T='place',W1='the')

0.0015748031496062992

In [209]:
Ptw(T='company',W1='the')

0.08976377952755907

## 14 Given this, if the word 'the' is found in a name, what is the most likely type?

It is most likely going to show a higher proability if it is a movie after the given is the word 'the'

## 15. Is Ptw(T='movie'|W1='the') the same as Pwt(W1='the'|T='movie') the same? Why or why not?

In [210]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [211]:
Pwt(W1='the', T='movie')

0.09214308527626956

No they are not the same because the first one is given the word 'the' what is the proability it is a movie and there are only 5 types that the word could be associated to so it makes sense that it would have a higher probability than the latter. If you are given the word 'movie' there is much of a less probaility that the first word is the because there are way more words in the column than there are types.

## 16. Do you think modeling Ptw(T|W1) would be better with a continuous function like a Gaussian? Why or why not?

- Answer in a markdown cell


I think it would be better to do a Gaussian if you were trying to visualize the data and figure out its shape. I think this would also be good to use the Gaussian becasue there can be a continuous amount of data being inputed, especially since there are so many different first words in the data set. It would be able to give us a good spread of the data.